In [1]:
from typing import NamedTuple
from pathlib import Path

import matplotlib.pyplot as plt

import torch
import numpy as np

from tools.iou_utils import get_max_iou

from cv2 import rectangle
from mmcv import Config
from mmcv.runner import wrap_fp16_model, load_checkpoint
from mmdet.models import build_detector
from mmcv.parallel import MMDataParallel
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

# PSTR's person search

This notebook uses PSTR model to illustrate the person search step-by-step.



## Load data, model and compute features.

Those are the line of codes needed to run PSTR on some hand-picked samples for illustration.

s9 s10 puis des autres

In [2]:
# Load configs file with boiler plate
cfg = Config.fromfile(Path("./configs/pstr/pstr_r50_24e_cuhk.py"))
cfg.model.pretrained = None
if cfg.model.get('neck'):
    if isinstance(cfg.model.neck, list):
        for neck_cfg in cfg.model.neck:
            if neck_cfg.get('rfp_backbone'):
                if neck_cfg.rfp_backbone.get('pretrained'):
                    neck_cfg.rfp_backbone.pretrained = None
    elif cfg.model.neck.get('rfp_backbone'):
        if cfg.model.neck.rfp_backbone.get('pretrained'):
            cfg.model.neck.rfp_backbone.pretrained = None
if isinstance(cfg.data.test, dict):
    cfg.data.test.test_mode = True
    samples_per_gpu = cfg.data.test.pop('samples_per_gpu', 1)
    if samples_per_gpu > 1:
        # Replace 'ImageToTensor' to 'DefaultFormatBundle'
        cfg.data.test.pipeline = replace_ImageToTensor(
            cfg.data.test.pipeline)
elif isinstance(cfg.data.test, list):
    for ds_cfg in cfg.data.test:
        ds_cfg.test_mode = True
    samples_per_gpu = max(
        [ds_cfg.pop('samples_per_gpu', 1) for ds_cfg in cfg.data.test])
    if samples_per_gpu > 1:
        for ds_cfg in cfg.data.test:
            ds_cfg.pipeline = replace_ImageToTensor(ds_cfg.pipeline)
cfg.model.train_cfg = None

In [3]:
# Get pedes_dataset
pedes_dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
    [ pedes_dataset[i] for i in range(100+1)],
    samples_per_gpu=1,
    workers_per_gpu=1,
    dist=False,
    shuffle=False)

loading annotations into memory...
Done (t=0.22s)
creating index...
index created!


In [ ]:
b = next(iter(data_loader))
b_img, b_img_metas = (b['img'], b['img_metas'])

## img

In [5]:
type(b_img), len(b_img), b_img[0].shape, 

(list, 1, torch.Size([1, 3, 844, 1500]))

## img_metas

In [6]:
b_img_metas

[DataContainer([[{'filename': '/sysu/Image/SSM/s15535.jpg', 'ori_filename': 's15535.jpg', 'ori_shape': (450, 800, 3), 'img_shape': (844, 1500, 3), 'pad_shape': (844, 1500, 3), 'scale_factor': array([1.875    , 1.8755555, 1.875    , 1.8755555], dtype=float32), 'flip': False, 'flip_direction': None, 'img_norm_cfg': {'mean': array([123.675, 116.28 , 103.53 ], dtype=float32), 'std': array([58.395, 57.12 , 57.375], dtype=float32), 'to_rgb': True}}]])]

In [23]:
(
    len(b_img_metas), 
    type(b_img_metas[0]), 
    type(b_img_metas[0].data), 
    len(b_img_metas[0].data),
    type(b_img_metas[0].data[0]),
    len(b_img_metas[0].data[0]),
    type(b_img_metas[0].data[0][0]),
)

(1, mmcv.parallel.data_container.DataContainer, list, 1, list, 1, dict)

In [25]:
b_img_metas[0].data[0][0]

{'filename': '/sysu/Image/SSM/s15535.jpg',
 'ori_filename': 's15535.jpg',
 'ori_shape': (450, 800, 3),
 'img_shape': (844, 1500, 3),
 'pad_shape': (844, 1500, 3),
 'scale_factor': array([1.875    , 1.8755555, 1.875    , 1.8755555], dtype=float32),
 'flip': False,
 'flip_direction': None,
 'img_norm_cfg': {'mean': array([123.675, 116.28 , 103.53 ], dtype=float32),
  'std': array([58.395, 57.12 , 57.375], dtype=float32),
  'to_rgb': True}}